In [5]:
from sdid import sdid
from get_data import quota, california_prop99

In [8]:
a = sdid(california_prop99(), "State", "Year", "treated", "PacksPerCapita")
print(a)


[12.] 12
-15.60382792084103


In [7]:
b = sdid(quota(), "country", "year", "quota", "womparl")
print(b)

[16. 28. 26. 11.  6.  4.  3.] 94
8.034018275508096
